In [95]:
import time
import base64
import random
import secrets
import hashlib
import string
import numpy as np
from collections import Counter, defaultdict
import heapq
from sympy import nextprime
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives import hashes


In [97]:
# Node class for Huffman Tree
class Node:
    def __init__(self, char=None, freq=None):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None

    # For priority queue
    def __lt__(self, other):
        return self.freq < other.freq


In [99]:
# 1. Build Huffman Tree
def build_huffman_tree(text):
    freq = Counter(text)
    heap = [Node(char, freq) for char, freq in freq.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)

        merged = Node(None, left.freq + right.freq)
        merged.left = left
        merged.right = right
        heapq.heappush(heap, merged)

    return heap[0] if heap else None


# 2. Generate Huffman Codes
def generate_huffman_codes(root):
    codes = {}

    def traverse(node, code=""):
        if node is None:
            return
        if node.char is not None:
            codes[node.char] = code
            return
        traverse(node.left, code + "0")
        traverse(node.right, code + "1")

    traverse(root)
    return codes


# 3. Encode text using Huffman Codes
def huffman_encode(text, codes):
    return "".join(codes[ch] for ch in text)

In [101]:
def xor_with_key(binary_text, key):
    '''missing_padding = len(key) % 4
    if missing_padding:
        key += '=' * (4 - missing_padding)

    key_bytes = base64.b64decode(key) 
    key_bin = ''.join(format(byte, '08b') for byte in key_bytes)

    key_bin = (key_bin * (len(binary_text) // len(key_bin) + 1))[:len(binary_text)]
    return ''.join(str(int(b) ^ int(k)) for b, k in zip(binary_text, key_bin))
'''
def xor_with_key(binary_text, key):
    # Ensure the key is repeated or truncated to match the length of the binary text
    key_bin = (key * (len(binary_text) // len(key) + 1))[:len(binary_text)]
    return ''.join(str(int(b) ^ int(k)) for b, k in zip(binary_text, key_bin))


In [103]:
#DNA Encoding
def binary_to_dna(binary_str):
    dna_mapping = {"00": "A", "01": "T", "10": "G", "11": "C"}
    padded_binary = binary_str + "0" * ((4 - len(binary_str) % 4) % 4)
    dna_encoded = "".join(dna_mapping[padded_binary[i:i+2]] for i in range(0, len(padded_binary), 2))
    return dna_encoded, len(padded_binary) - len(binary_str)

def dna_to_rna(dna_str):
    return dna_str.replace("T", "U")

In [105]:
def adaptive_permutation(text, key):
    seed = sum(ord(c) for c in key) % 256
    np.random.seed(seed)
    indices = np.random.permutation(len(text))
    return ''.join(text[i] for i in indices)

In [107]:
from Bio.Seq import Seq
from Bio.Data import CodonTable


standard_codon_table = CodonTable.unambiguous_rna_by_name["Standard"]

def rna_to_protein(rna_sequence):
    padding = (3 - len(rna_sequence) % 3) % 3
    padded_rna = rna_sequence + "A" * padding  # Use 'A' as neutral padding
    
    print(f" RNA Before Protein Translation: {padded_rna} (Padding: {padding})")

    codons = [padded_rna[i:i+3] for i in range(0, len(padded_rna), 3)]
    
    # Translate RNA to Protein
    protein_seq = str(Seq(padded_rna).translate(to_stop=False))

    print(f"Translated Protein: {protein_seq}")
    print(f" Codon Map: {codons}")
    
    return protein_seq, padding, codons

def protein_to_rna(protein_seq, padding, codon_map):
    print(f" Protein Before RNA Translation: {protein_seq}")

    rna_sequence = "".join(codon_map[i] for i in range(len(protein_seq)))

    if padding:
        rna_sequence = rna_sequence[:-padding]

    print(f" RNA After Reverse Translation: {rna_sequence}")
    return rna_sequence


In [109]:
def encrypt(plain_text, key):
    print("Encryption")

    #Huffman Encoding
    huff_tree = build_huffman_tree(plain_text)
    huff_codes = generate_huffman_codes(huff_tree)
    huffman_encoded = huffman_encode(plain_text, huff_codes)
    print(f"Huffman Encoded Binary: {huffman_encoded}")
    
    #XOR Encryption
    xor_result = xor_with_key(huffman_encoded, key)
    print(f"After XOR Encryption: {xor_result}")

    #DNA Encoding
    dna_encoded, dna_padding = binary_to_dna(xor_result)
    print(f"After DNA Encoding: {dna_encoded}")

    #RNA Conversion
    rna_encoded = dna_to_rna(dna_encoded)
    print(f"After RNA Encoding: {rna_encoded}")

    #Adaptive Permutation
    permuted_text = adaptive_permutation(rna_encoded, key)
    print(f"After Permutation: {permuted_text}")

    # Step 6: RNA to Protein Conversion
    protein_seq, protein_padding ,codon_map= rna_to_protein(permuted_text)
    print(f"Protein Sequence: {protein_seq} ")
    
    return protein_seq, huff_codes, dna_padding, protein_padding,codon_map


In [111]:
def rna_to_dna(rna_str):
    return rna_str.replace("U", "T")
def dna_to_binary(dna_str):
    dna_mapping_reverse = {"A": "00", "T": "01", "G": "10", "C": "11"}
    binary_str = "".join(dna_mapping_reverse[base] for base in dna_str)
    return binary_str

In [113]:
def reverse_adaptive_permutation(permuted_text, key):
    seed = sum(ord(c) for c in key) % 256
    np.random.seed(seed)
    indices = np.random.permutation(len(permuted_text))
    original_order = np.argsort(indices)
    return ''.join(permuted_text[i] for i in original_order)

In [115]:
def decrypt(permuted_text, key, huffman_codes, dna_padding, protein_padding):
    print("Starting Decryption...")

    rna_text = protein_to_rna(permuted_text, protein_padding, codon_map)
    print(f"After Protein to RNA Conversion: {rna_text}")

    reversed_text = reverse_adaptive_permutation(rna_text, key)
    print(f"After Reverse Permutation: {reversed_text}")

    dna_text = rna_to_dna(reversed_text)
    print(f"After RNA to DNA Conversion: {dna_text}")

    binary_text = dna_to_binary(dna_text)
    print(f"After DNA to Binary Conversion: {binary_text}")

    # Step 5: Remove Padding
    original_length = len(binary_text)
    if dna_padding:
        binary_text = binary_text[:-(dna_padding)]  
    print(f" Binary Length Before Padding Removal: {original_length}")
    print(f" Binary Length After Padding Removal: {len(binary_text)}")
    print(f" After Removing Padding: {binary_text}")

    # Step 6: XOR Decryption
    original_xor = xor_with_key(binary_text, key)
    print(f"After XOR Decryption: {original_xor}")

    # Step 7: Huffman Decoding
    inverted_huffman_codes = {v: k for k, v in huffman_codes.items()}
    decoded_text = ""
    buffer = ""
    for bit in original_xor:
        buffer += bit
        if buffer in inverted_huffman_codes:
            decoded_text += inverted_huffman_codes[buffer]
            print(f"Decoded Character: {inverted_huffman_codes[buffer]} from Buffer: {buffer}")
            buffer = ""

    if buffer:
        print(f" Huffman decoding failed! Buffer left: {buffer}")
        print(f"Incomplete Huffman Decoding: {decoded_text}")

    print(f"Decrypted Text: {decoded_text}")
    print(f"Decryption Time: {time.time() - start_time:.6f} seconds\n")

    return decoded_text


In [117]:
def generate_ecdh_keys():
    private_key = ec.generate_private_key(ec.SECP256R1())  # NIST P-256 curve
    public_key = private_key.public_key()
    return private_key, public_key

def derive_shared_secret(private_key, peer_public_key):
    shared_secret = private_key.exchange(ec.ECDH(), peer_public_key)
    return hashlib.sha256(shared_secret).digest()
def generate_chaotic_key(length):
    chars = string.ascii_letters + string.digits + string.punctuation
    return ''.join(random.choice(chars) for _ in range(length))

# --- Main Logic ---
'''private_key_A, public_key_A = generate_ecdh_keys()
private_key_B, public_key_B = generate_ecdh_keys()

shared_secret_A = derive_shared_secret(private_key_A, public_key_B)
shared_secret_B = derive_shared_secret(private_key_B, public_key_A)

assert shared_secret_A == shared_secret_B

chaotic_key = generate_chaotic_key(256)

shared_secret_b64 = base64.b64encode(shared_secret_A).decode()
final_key = shared_secret_b64[:len(chaotic_key)] + chaotic_key[:len(shared_secret_b64)]
final_key = final_key[:2]
print("Final 16-bit Key:", final_key.encode('utf-8'))'''
final_key="1011101011101110"




In [119]:
def read_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()
def write_to_file(file_path, data):
    with open(file_path, 'w') as file:
        file.write(data)
input_path = "input.txt"
output_path = "output.txt"
plain_text = read_file(input_path)

start_time = time.time()
encrypted, huffman_codes, dna_padding, protein_padding, codon_map = encrypt(plain_text, final_key)
end_time = time.time()

print(f"Encryption Time: {end_time - start_time:.6f} seconds")

write_to_file(output_path, encrypted)
print(f"Encrypted text saved to: {output_path}")


Encryption
Huffman Encoded Binary: 11100001010110111101111001010001
After XOR Encryption: 01011011101101010110010010111111
After DNA Encoding: TTGCGCTTTGTAGCCC
After RNA Encoding: UUGCGCUUUGUAGCCC
After Permutation: UCUUCGCCGCGUUAUG
 RNA Before Protein Translation: UCUUCGCCGCGUUAUGAA (Padding: 2)
Translated Protein: SSPRYE
 Codon Map: ['UCU', 'UCG', 'CCG', 'CGU', 'UAU', 'GAA']
Protein Sequence: SSPRYE 
Encryption Time: 0.002556 seconds
Encrypted text saved to: output.txt


In [121]:
def write_to_file(file_path, data):
    with open(file_path, 'w') as file:
        file.write(data)
decrypted_output_path = "output.txt"
start_time = time.time()
decrypted = decrypt(encrypted, final_key, huffman_codes, dna_padding, protein_padding)
end_time = time.time()

print(f"Decryption Time: {end_time - start_time:.6f} seconds")
write_to_file(decrypted_output_path, decrypted)
print(f"Decrypted text saved to: {decrypted_output_path}")


Starting Decryption...
 Protein Before RNA Translation: SSPRYE
 RNA After Reverse Translation: UCUUCGCCGCGUUAUG
After Protein to RNA Conversion: UCUUCGCCGCGUUAUG
After Reverse Permutation: UUGCGCUUUGUAGCCC
After RNA to DNA Conversion: TTGCGCTTTGTAGCCC
After DNA to Binary Conversion: 01011011101101010110010010111111
 Binary Length Before Padding Removal: 32
 Binary Length After Padding Removal: 32
 After Removing Padding: 01011011101101010110010010111111
After XOR Decryption: 11100001010110111101111001010001
Decoded Character: H from Buffer: 1110
Decoded Character: e from Buffer: 000
Decoded Character: l from Buffer: 10
Decoded Character: l from Buffer: 10
Decoded Character: o from Buffer: 110
Decoded Character:   from Buffer: 1111
Decoded Character: W from Buffer: 011
Decoded Character: o from Buffer: 110
Decoded Character: r from Buffer: 010
Decoded Character: l from Buffer: 10
Decoded Character: d from Buffer: 001
Decrypted Text: Hello World
Decryption Time: 0.001000 seconds

Decrypt

In [123]:
assert decrypted == plain_text, "Decryption failed!"
print("\nEncryption and Decryption Successful!")


Encryption and Decryption Successful!


In [125]:
import random
import string
def generate_random_string(length=100
                          ):
    characters = string.ascii_letters + string.digits
    return ''.join(random.choice(characters) for _ in range(length))

file_path = 'input.txt'
random_string = generate_random_string(10000)
with open(file_path, 'w') as file:
    file.write(random_string)
